In [ ]:
import ants
import numpy as np

In [ ]:
fname1 = ants.get_ants_data('r16')
fname2 = ants.get_ants_data('r64')
print(fname1)
print(fname2)

In [ ]:
img1 = ants.image_read(fname1)
img2 = ants.image_read(fname2)
print(img1)

In [ ]:
arr_4d = np.random.randn(70,70,70,10).astype('float32')
img_fmri = ants.from_numpy(arr_4d, has_components=True)
print(img_fmri)

# todo : tester aussi from_nibabel/ to_nibabel (-> .nii.gz)
#        nifti_to_ants

In [ ]:
img = ants.image_read(ants.get_ants_data('r16'))
img = ants.resample_image(img, (64,64), 1, 0)
mask = ants.get_mask(img)
img_seg = ants.atropos(a=img, 
                       m='[0.2,1x1]', 
                       c='[2,0]', 
                       i='kmeans[3]', 
                       x=mask)
print(img_seg.keys())
ants.plot(img_seg['segmentation'])

In [ ]:
mg = ants.image_read( ants.get_ants_data('r16') ,2)
mask = ants.get_mask( img ).threshold_image( 1, 2 )
segs=ants.atropos( a = img, 
                   m = '[0.2,1x1]', 
                   c = '[2,0]',  
                   i = 'kmeans[3]', 
                   x = mask )
thickimg = ants.kelly_kapowski(s=segs['segmentation'], 
                               g=segs['probabilityimages'][1],
                               w=segs['probabilityimages'][2], 
                               its=45, 
                               r=0.5, 
                               m=1)
print(thickimg)
img.plot(overlay=thickimg, overlay_cmap='jet')

# Transform manually template 

In [ ]:
import nrrd
import nibabel as nib
import numpy as np
import os
from pathlib import Path

template_file = '/Users/mahdi/stage-2022-mahdi/src/allen_template_25.nii.gz'
template_ras_file = "/Users/mahdi/stage-2022-mahdi/src/allen_template_25_manualRAS.nii.gz"

resolution = 25
res_avgt_mm = 70 / 1e3
avgt_offset = np.array([-5.675, -8.79448, -8.450335, 0])

# Load and manually transform the template
vol = nib.load(template_file).get_fdata()

vol_ras = np.moveaxis(vol, (0,1,2), (1,2,0))
vol_ras = np.flip(vol_ras, axis=2)
vol_ras = np.flip(vol_ras, axis=1)

affine = np.eye(4) * res_avgt_mm

img_ras = nib.Nifti1Image(vol_ras, affine)
nib.save(img_ras, template_ras_file)

# load reference
avgt = nib.load('/Users/mahdi/stage-2022-mahdi/src/AVGT.nii.gz')
ref_vol = avgt.get_fdata()

# load moving
allen = nib.load('/Users/mahdi/stage-2022-mahdi/src/allen_template_25_manualRAS.nii.gz')
allen_vol = allen.get_fdata()


allen_vol = allen_vol.astype(float)
ref_vol = ref_vol.astype(float)

fixed = ants.from_numpy( ref_vol ).resample_image((164, 212, 158),1,0)
moving = ants.from_numpy( allen_vol ).resample_image((164, 212, 158),1,0)

fixed.plot_ortho()
moving.plot_ortho()
mytx = ants.registration(fixed=fixed , moving=moving, type_of_transform='SyN' )

warped_moving = ants.apply_transforms(fixed=fixed, moving=moving, 
                                          transformlist=mytx['fwdtransforms'])
fixed.plot_ortho_double(warped_moving)
affine_offset = affine.copy()
affine_offset[:,3] + avgt_offset

warped_vol = warped_moving.numpy()
# Saving the volume (nifti)
img = nib.Nifti1Image(warped_vol, affine_offset)
nib.save(img, '/Users/mahdi/stage-2022-mahdi/src/template_reg.nii.gz')

In [ ]:
warped_moving.shape

### essais nii nibabel ants

In [ ]:
# todo : tester aussi from_nibabel/ to_nibabel (-> .nii.gz)
#        nifti_to_ants

import ants
import nrrd
import numpy as np

array, metadata = nrrd.read('/Users/mahdi/stage-2022-mahdi/src/average_template_25.nrrd')

img = ants.from_numpy(array) # convertir en float

orientation = ants.get_orientation(img)

print(orientation)

In [ ]:
import ants
import numpy as np
import matplotlib.pyplot as plt

img = ants.image_read(ants.get_ants_data('r16'))
mask = ants.get_mask(img, cleanup=15, low_thresh=55)
ants.plot(mask)
nimg = img * mask
ants.plot(nimg,black_bg=False)


In [ ]:
import numpy as np
import nibabel as nib


ants.ANTsImage.pixeltype = np.int32
img = nib.load('/Users/mahdi/stage-2022-mahdi/src/288169135_projection_density_50.nii.gz')

vol = img.get_fdata().astype(np.float32)

img = ants.from_numpy( vol ).resample_image((164, 212, 158),1,0)

img.plot_ortho()

for z in range(158):
    for y in range(212):
        for x in range(164):
            if vol[x, y, z] < 1e-01:
                vol[x, y, z] = 0.0

img = ants.from_numpy( vol )

img.plot_ortho()

 # Removing background 
    # mask = ants.get_mask(warped_moving)
    # warped_moving = warped_moving * mask



#####BINARIZED FORMULA######    S
    #warped_moving = warped_moving > 1e-01

In [ ]:
import cv2

tmp = cv2.cvtColor(img.numpy(), cv2.COLOR_BGR2GRAY)
_,alpha = cv2.threshold(tmp,0,255,cv2.THRESH_BINARY)
b, g, r = cv2.split(img.numpy())
rgba = [b,g,r, alpha]
dst = cv2.merge(rgba,4)


In [ ]:
from PIL import Image
import numpy as np

image = Image.open('../images/288169135_proj_density_/288169266.jpg')

vol = np.asarray(image)

img = nib.Nifti1Image(vol, np.eye(4))

nib.save(img, '../../niftis/test.nii')

# test fusions 2 niftis

In [1]:
import ants
import numpy as np
import nibabel as nib
import PIL

In [17]:
vol = nib.load('/Users/mahdi/stage-2022-mahdi/niftis/485847695_AIp_R_proj_density_100.nii.gz').get_fdata()
voll = nib.load('/Users/mahdi/stage-2022-mahdi/niftis/180403712_ECT_R_proj_density_100.nii.gz').get_fdata()

In [11]:
print(vol[109,97,117],
      voll[109,97,117])   

0.0 0.0004686765605583787


In [19]:
print(vol[117,69,76],
      voll[117,69,76])

0.00012096612044842914 0.0014684608904644847


In [18]:
rgb = np.zeros((164,212,158,1,1), [('R', 'u1'), ('G','u1'), ('B', 'u1'), ('A', 'u1')])

for i in range(164):
    for j in range(212):
        for k in range(158):
            if vol[i,j,k] == 0.0 and voll[i,j,k] == 0.0:
                rgb[i,j,k] = (0, 0, 0, 0)
            else:
                rgb[i,j,k] = (vol[i,j,k] * 255, voll[i,j,k] * 255, 0, 255)

img_ = nib.Nifti1Image(rgb, affine)
nib.save(img_, '/Users/mahdi/stage-2022-mahdi/niftis/rbg_180601025_183171679.nii.gz')

In [21]:
print(rgb[117,69,76])

[[(0, 0, 0, 255)]]


In [3]:
vol1 = nib.load('/Users/mahdi/stage-2022-mahdi/niftis/288169135_MOp_R_proj_density_25.nii.gz').get_fdata()
vol2 = nib.load('/Users/mahdi/stage-2022-mahdi/niftis/100141563_MOp_R_proj_density_25.nii.gz').get_fdata()
vol3 = nib.load('/Users/mahdi/stage-2022-mahdi/niftis/603468246_ENTm_L_proj_density_25.nii.gz').get_fdata()

v4 = nib.load('/Users/mahdi/Downloads/180601025_SSp-un_R_proj_density_25.nii.gz').get_fdata()
v5 = nib.load('/Users/mahdi/Downloads/183171679_SSp-bfd_R_proj_density_25.nii.gz').get_fdata()

In [15]:
affine = nib.load('/Users/mahdi/stage-2022-mahdi/utils/AVGT.nii.gz').affine

In [4]:
rgb = np.zeros((164,212,158,1,1), [('R', 'u1'), ('G','u1'), ('B', 'u1'), ('A', 'u1')])

for i in range(164):
    for j in range(212):
        for k in range(158):
            if v4[i,j,k] == 0 and v5[i,j,k] == 0:
                rgb[i,j,k] = (0, 0, 0, 0)
            else:
                rgb[i,j,k] = (v5[i,j,k] * 255, v4[i,j,k] * 255, 0, 255)

img_ = nib.Nifti1Image(rgb, affine)
nib.save(img_, '/Users/mahdi/stage-2022-mahdi/niftis/rbg_180601025_183171679.nii.gz')

## Test creation volume RGB

In [5]:
arr = np.zeros((164,212,158,1,1), [('R', 'u1'), ('G','u1'), ('B', 'u1'), ('A', 'u1')])

for i in range(164):
    for j in range(212):
        for k in range(158):
            if vol1[i,j,k] == 0 and vol2[i,j,k] == 0:
                arr[i,j,k] = (0, 0, 0, 0)
            else:
                arr[i,j,k] = (vol1[i,j,k] * 255, vol2[i,j,k] * 255, 0, 255)


In [6]:
print(arr[101,127,118,0,0])
print(arr[51,103,65,0,0])

(255, 62, 0, 255)
(0, 0, 0, 0)


In [7]:
img = nib.Nifti1Image(arr, affine)
nib.save(img, '/Users/mahdi/stage-2022-mahdi/niftis/rbg.nii.gz')

In [150]:
fname = '/Users/mahdi/stage-2022-mahdi/niftis/rbg.nii.gz'
im = nib.load(fname)
print(im.get_data_dtype())
print(im.header)

[('R', 'u1'), ('G', 'u1'), ('B', 'u1'), ('A', 'u1')]
<class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : b''
db_name         : b''
extents         : 0
session_error   : 0
regular         : b''
dim_info        : 0
dim             : [  5 164 212 158   1   1   1   1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : RGBA
bitpix          : 32
slice_start     : 0
pixdim          : [1.         0.06999999 0.06999999 0.06999999 1.         1.
 1.         1.        ]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_code      : unknown
xyzt_units      : 0
cal_max         : 0.0
cal_min         : 0.0
slice_duration  : 0.0
toffset         : 0.0
glmax           : 0
glmin           : 0
descrip         : b''
aux_file        : b''
qform_code      : unknown
sform_code      : aligned
quatern_b       : 0.0
quatern_c       : 0.0
quatern_d       : 0.0
qoffset_x     

# Mosaic

In [2]:
import ants
import numpy as np
import nibabel as nib

In [3]:
template = nib.load('/Users/mahdi/stage-2022-mahdi/niftis/allen_volume_100.nii.gz')
xroi_mask = nib.load('/Users/mahdi/stage-2022-mahdi/niftis/485847695_180403712_crossing_25/485847695_180403712_x-rois_mask_25.nii.gz')

In [4]:
template_arr = template.get_fdata()
xroi_mask_arr = xroi_mask.get_fdata()

In [14]:
xroi = (template_arr * xroi_mask_arr).clip(0,255).astype(np.uint8)

In [17]:
xroi_img = nib.Nifti1Image(xroi, affine)
nib.save(xroi_img, "/Users/mahdi/stage-2022-mahdi/niftis/xroi.nii.gz")

# Tracto to avgt

In [1]:
import ants
import numpy as np
import nibabel as nib
import nibabel.streamlines as nis
from dipy.io.streamline import load_tractogram, save_tractogram

In [2]:
avgt = nib.load('/Users/mahdi/stage-2022-mahdi/AVGT.nii.gz')
affine = avgt.affine
ref_vol = avgt.get_fdata()
shape = avgt.shape

In [444]:
print(affine)

[[ 0.06999999  0.          0.         -5.67500019]
 [ 0.          0.06999999  0.         -8.79448032]
 [ 0.          0.          0.06999999 -8.45033455]
 [ 0.          0.          0.          1.        ]]


In [154]:
tract = load_tractogram('/Users/mahdi/stage-2022-mahdi/niftis/allen_wildtype_tractogram.tck')

In [155]:
data_mm_allen = tract[0].get_data()

In [156]:
B = np.array([[ 0.05,  0,        0,      0],
             [  0,    -0.05,     0,      0],
             [  0,     0,       -0.05,   0],
             [  0,     0,        0,      1]])

In [157]:
data_vox_allen = []

In [160]:
for point in data_mm_allen:
    point_mm = np.concatenate((point, np.ndarray(1)))
    point_voxel = (np.linalg.inv(B) @ point_mm)
    point = point_voxel[0:3]
    data_vox_allen.append(point)

In [161]:
print(data_mm_allen[1], data_vox_allen[1])

[ 4.40700006 -5.55600023 -6.98899984] [  88.1400013   111.12000465  139.77999687]


In [162]:
data_vox_avgt = []

In [163]:
tx25 = ants.read_transform('/Users/mahdi/stage-2022-mahdi/transformations_allen2avgt/allen2avgtAffine_50.mat')
itx25 = tx25.invert()

In [164]:
print(itx25.apply_to_point([139.77999687, -175.8599987,    48.87999535 ]))

(99.83096313476562, -101.90909576416016, 3.2597193717956543)


In [165]:
def pir_ras(coords, pir):
    p, i, r = pir[0], pir[1], pir[2]
    x, y, z = coords[0], coords[1], coords[2]
    x_, y_, z_ = z, p-x, i-y
    return [x_, y_, z_]

In [166]:
for point in data_vox_allen:
    allen_ras = pir_ras([point[0], point[1], point[2]], (264,160,228))
    point = itx25.apply_to_point(allen_ras)
    pt = [point[0], point[1], point[2]]
    data_vox_avgt.append(pt)

In [167]:
print(len(data_vox_avgt), data_vox_avgt[1])

26551916 [99.82926940917969, 141.4864501953125, 65.3900375366211]


In [168]:
data_mm_avgt = []

In [169]:
A = np.array([[-0.07,  0,        0,      5.675  ],
             [  0,    -0.07,     0,      8.79448],
             [  0,     0,        0.07,  -8.40533],
             [  0,     0,        0,      1      ]])

In [170]:
def mibrain_voxel2mm_RAS(coords_voxel, A):
    x, y, z = coords_voxel[0], coords_voxel[1], coords_voxel[2]
    coords_voxel = np.array([x, y, z, 1])
    coords_mm = A @ coords_voxel

    return coords_mm[0:3]


In [171]:
for point in data_vox_avgt:
    mm = mibrain_voxel2mm_RAS(point, A)
    data_mm_avgt.append(mm)

In [172]:
print(data_mm_allen[1], data_vox_allen[1], data_vox_avgt[1], data_mm_avgt[1])

[ 4.40700006 -5.55600023 -6.98899984] [  88.1400013   111.12000465  139.77999687] [99.82926940917969, 141.4864501953125, 65.3900375366211] [-1.31304886 -1.10957151 -3.82802737]


In [173]:
array_sequence = nis.ArraySequence(np.array([data_mm_avgt]))

In [174]:
save_tractogram(
    fname="/Users/mahdi/stage-2022-mahdi/niftis/tracto_avgt.trk",
    streamlines=array_sequence,
    affine=np.eye(4),
    )

In [41]:
avgt_tract = load_tractogram('/Users/mahdi/stage-2022-mahdi/niftis/tracto_avgt.tck')

In [42]:
print(avgt_tract)

(ArraySequence([array([[-5.76068354, -8.87338352, -8.72115612],
       [-5.76691341, -8.87215042, -8.71829605],
       [-5.76663446, -8.86604214, -8.71473312],
       ..., 
       [-5.40215921, -8.64045048, -8.84667206],
       [-5.3993597 , -8.64526844, -8.84205055],
       [-5.39663124, -8.65155411, -8.83525085]], dtype=float32)]), {'count': '0000000001', 'datatype': 'Float32LE', 'voxel_sizes': '(0.07, 0.07, 0.07)', 'dimensions': '(164, 212, 158)', 'voxel_order': 'RAS', 'file': '. 145', 'magic_number': b'mrtrix tracks', 'endianness': '<', '_dtype': dtype('float32'), '_offset_data': 145, 'nb_streamlines': 1, 'voxel_to_rasmm': array([[ 1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.],
       [ 0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  1.]])})


In [43]:
print(tract)

(ArraySequence([array([[ 4.4000001 , -5.5999999 , -6.9000001 ],
       [ 4.40700006, -5.55600023, -6.98899984],
       [ 4.47900009, -5.48500013, -6.98500013],
       [ 4.48100042, -5.40899992, -6.99900007],
       [ 4.51600027, -5.40700006, -6.99800062],
       [ 4.57999992, -5.40800047, -7.0250001 ],
       [ 4.57700014, -5.37599993, -7.00400019],
       [ 4.59700012, -5.39599991, -7.0250001 ],
       [ 4.59200001, -5.36300039, -7.00699997],
       [ 4.5999999 , -5.32599974, -7.00600004],
       [ 4.62000036, -5.34600019, -7.0260005 ],
       [ 4.5999999 , -5.32599974, -7.00600004],
       [ 4.62000036, -5.34600019, -7.0260005 ],
       [ 4.5999999 , -5.32599974, -7.00600004],
       [ 4.61000013, -5.28700018, -7.00800037],
       [ 4.61000013, -5.24500036, -6.99800062],
       [ 4.63700008, -5.19399977, -7.02200031],
       [ 4.68300009, -5.125     , -7.00699997],
       [ 4.73000002, -5.07999992, -7.02200031],
       [ 4.70900011, -5.0619998 , -7.00199986],
       [ 4.73000002, -5.

In [29]:
trkavgt = load_tractogram('/Users/mahdi/stage-2022-mahdi/niftis/allen_wildtype_tractogram_avgt.trk')

In [62]:
H = np.array([[ 0.06999999,  0.        ,  0.        ,   -5.67500019   ],
              [ 0.        ,  0.06999999,  0.        ,   -8.79448032 ],
              [ 0.        ,  0.        ,  0.06999999,   -8.45033455],
              [ 0.        ,  0.        ,  0.        ,  1.           ]])

In [91]:
print(np.linalg.inv(H) @ [ 0.48313593  , 3.57944513 , -2.86387486  , 1.        ])

[  87.97338571  176.77038882   79.8065784     1.        ]


In [87]:
aff = np.array([[  1.40010476e+00,    7.93613726e-05 ,   2.72541976e-04 ,  -5.67500019], 
                [ -1.61930133e-04 ,   1.35685694e+00 ,  -3.45518261e-01 ,  8.79448032], 
                [ -1.27313469e-04 ,   3.46717477e-01,    1.35826540e+00 ,   8.45033455], 
                [  0.00000000e+00 ,   0.00000000e+00 ,   0.00000000e+00,    1.00000000e+00]])

In [89]:
print( aff @[ 4.4000001 , -5.5999999 , -6.9000001, 1])

[ 0.48313593  3.57944513 -2.86387486  1.        ]


In [30]:
print(trkavgt)

(ArraySequence([array([[  6.14295292e+00,   3.34790726e+01,  -2.20813036e+00],
       [  6.15273380e+00,   3.35695267e+01,  -2.31376123e+00],
       [  6.25354671e+00,   3.36644669e+01,  -2.28372049e+00],
       [  6.25635052e+00,   3.37724228e+01,  -2.27638578e+00],
       [  6.30535507e+00,   3.37747879e+01,  -2.27433968e+00],
       [  6.39495277e+00,   3.37827492e+01,  -2.31136703e+00],
       [  6.39076138e+00,   3.38189125e+01,  -2.27174759e+00],
       [  6.41875744e+00,   3.37990303e+01,  -2.30720806e+00],
       [  6.41176319e+00,   3.38375893e+01,  -2.27131701e+00],
       [  6.42296696e+00,   3.38874435e+01,  -2.25713158e+00],
       [  6.45096302e+00,   3.38672142e+01,  -2.29123402e+00],
       [  6.42296696e+00,   3.38874435e+01,  -2.25713158e+00],
       [  6.45096302e+00,   3.38672142e+01,  -2.29123402e+00],
       [  6.42296696e+00,   3.38874435e+01,  -2.25713158e+00],
       [  6.43697166e+00,   3.39410477e+01,  -2.24632788e+00],
       [  6.43697643e+00,   3.39945831e

In [27]:
trk = load_tractogram('/Users/mahdi/stage-2022-mahdi/niftis/allen_wildtype_tractogram.trk')

In [28]:
print(trk)

(ArraySequence([array([[ 4.4000001 , -5.5999999 , -6.9000001 ],
       [ 4.40700006, -5.55600023, -6.98899984],
       [ 4.47900009, -5.48500013, -6.98500013],
       [ 4.48100042, -5.40899992, -6.99900007],
       [ 4.51600027, -5.40700006, -6.99800062],
       [ 4.57999992, -5.40800047, -7.0250001 ],
       [ 4.57700014, -5.37599993, -7.00400019],
       [ 4.59700012, -5.39599991, -7.0250001 ],
       [ 4.59200001, -5.36300039, -7.00699997],
       [ 4.5999999 , -5.32599974, -7.00600004],
       [ 4.62000036, -5.34600019, -7.0260005 ],
       [ 4.5999999 , -5.32599974, -7.00600004],
       [ 4.62000036, -5.34600019, -7.0260005 ],
       [ 4.5999999 , -5.32599974, -7.00600004],
       [ 4.61000013, -5.28700018, -7.00800037],
       [ 4.61000013, -5.24500036, -6.99800062],
       [ 4.63700008, -5.19399977, -7.02200031],
       [ 4.68300009, -5.125     , -7.00699997],
       [ 4.73000002, -5.07999992, -7.02200031],
       [ 4.70900011, -5.0619998 , -7.00199986],
       [ 4.73000002, -5.

In [6]:
from scipy.io import loadmat, savemat

In [92]:
mat = loadmat('/Users/mahdi/stage-2022-mahdi/transformations_allen2avgt/allen2avgtAffine_50.mat')
affn = np.eye(4)
affn[:3, :3] = mat['AffineTransform_float_3_3'][:9].reshape((3,3))
affn[:3, 3] =  mat['AffineTransform_float_3_3'][9:].squeeze()

In [93]:
print(affn)

[[  1.40010476e+00   7.93613726e-05  -2.72541976e-04   3.25529709e+01]
 [ -1.61930133e-04   1.35685694e+00   3.45518261e-01   2.24906101e+01]
 [  1.27313469e-04  -3.46717477e-01   1.35826540e+00   4.84793854e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]]


In [22]:
data = mat['AffineTransform_float_3_3']
fixed = mat['fixed']

In [23]:
mat_dic = {
    'AffineTransform_double_3_3': data,
    'fixed': fixed
}

In [24]:
print(mat_dic)

{'AffineTransform_double_3_3': array([[  1.40010476e+00],
       [  7.93613726e-05],
       [ -2.72541976e-04],
       [ -1.61930133e-04],
       [  1.35685694e+00],
       [  3.45518261e-01],
       [  1.27313469e-04],
       [ -3.46717477e-01],
       [  1.35826540e+00],
       [  3.25529709e+01],
       [  2.24906101e+01],
       [  4.84793854e+00]], dtype=float32), 'fixed': array([[ 81.35860443],
       [ 94.48912811],
       [ 79.52913666]], dtype=float32)}


In [25]:
savemat(
    '/Users/mahdi/stage-2022-mahdi/transformations_allen2avgt/tract.mat',
    mat_dic)

In [8]:
print(mat)

{'AffineTransform_float_3_3': array([[  1.40010476e+00],
       [  7.93613726e-05],
       [ -2.72541976e-04],
       [ -1.61930133e-04],
       [  1.35685694e+00],
       [  3.45518261e-01],
       [  1.27313469e-04],
       [ -3.46717477e-01],
       [  1.35826540e+00],
       [  3.25529709e+01],
       [  2.24906101e+01],
       [  4.84793854e+00]], dtype=float32), 'fixed': array([[ 81.35860443],
       [ 94.48912811],
       [ 79.52913666]], dtype=float32)}


# dipy


In [3]:
from dipy.tracking.streamline import transform_streamlines

In [4]:
sft_file = '/Users/mahdi/stage-2022-mahdi/niftis/allen_wildtype_tractogram.tck'
ref = '/Users/mahdi/stage-2022-mahdi/allen_template_50.nii.gz'

In [7]:
transfo_dict = loadmat('/Users/mahdi/stage-2022-mahdi/transformations_allen2avgt/allen2avgtAffine_50.mat')
lps2ras = np.diag([-1, -1, 1])
rot = transfo_dict['AffineTransform_float_3_3'][0:9].reshape((3, 3))
trans = transfo_dict['AffineTransform_float_3_3'][9:12]
offset = transfo_dict['fixed']
r_trans = (np.dot(rot, offset) - offset - trans).T * [1, 1, -1]
data = np.eye(4)
data[0:3, 3] = r_trans
data[:3, :3] = np.dot(np.dot(lps2ras, rot), lps2ras)

In [8]:
print(data)

[[  1.40010476e+00   7.93613726e-05   2.72541976e-04  -1.51824951e-02]
 [ -1.61930133e-04   1.35685694e+00  -3.45518261e-01   3.86941071e+01]
 [ -1.27313469e-04   3.46717477e-01   1.35826540e+00   9.10607910e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]]


In [9]:
B = np.array([[ 0.05,  0,        0,      0],
             [  0,    -0.05,     0,      0],
             [  0,     0,       -0.05,   0],
             [  0,     0,        0,      1]])

In [10]:
B_ = np.array([[ -0.05,  0,        0,      0],
             [  0,    -0.05,     0,      0],
             [  0,     0,       -0.05,   0],
             [  0,     0,        0,      1]])

In [11]:
t = data @ np.eye(4)/2 @ np.linalg.inv(B)

In [12]:
t_ = data @ np.diag(np.array([.5, .5, .5, 1])) @ np.linalg.inv(B)

In [13]:
it = data @ np.eye(4)/2 @ np.linalg.inv(B_)

In [14]:
sft_streamlines, sft_hdr = load_tractogram(filename=sft_file)

In [15]:
warped_streamlines = transform_streamlines(sft_streamlines,t_)

In [16]:
new_sft = "/Users/mahdi/stage-2022-mahdi/niftis/new_tract.tck"

In [356]:
affine_copy = np.array([[ 0.7 , 0.        , 0.       , -1.49],
                       [ 0.         , 0.7, 0.        , -(8.37-5.60)+.4],
                       [ 0.         , 0.        , 0.7, -2.99+0.53-0.27-0.41-.3],
                       [ 0.         , 0.        , 0.        , 1]])

In [357]:
save_tractogram(
    new_sft,
    warped_streamlines,
    affine_copy
    )

In [368]:
save_tractogram(
    "/Users/mahdi/stage-2022-mahdi/niftis/testrack.tck",
    warped_streamlines,
    affine
    )

In [18]:
rotation = np.array([
    [1,  0,  0, 0],
    [0, -1,  0, 0],
    [0,  0, -1, 0],
    [0,  0,  0, 1]
    ])

In [19]:
translation = np.array([
    [1, 0, 0,  0  ],
    [0, 1, 0, -212],
    [0, 0, 1, -158],
    [0, 0, 0,  1  ]
    ])

In [458]:
save_tractogram(
    "/Users/mahdi/stage-2022-mahdi/niftis/test.tck",
    warped_streamlines,
    affine @ rotation @ translation
    )

In [22]:
save_tractogram(
    "/Users/mahdi/stage-2022-mahdi/niftis/tracktest.tck",
    warped_streamlines,
    affine @ rotation @ translation
    )

In [23]:
test = load_tractogram("/Users/mahdi/stage-2022-mahdi/niftis/test.tck")

In [24]:
print(test)

(ArraySequence([array([[-1.3615818 ,  1.04169774, -5.62821007],
       [-1.35470676,  1.10501516, -5.70215034],
       [-1.28414619,  1.17149186, -5.68110895],
       [-1.28218722,  1.24706304, -5.67597437],
       [-1.24788511,  1.24872482, -5.67453575],
       [-1.18515563,  1.25431204, -5.70044327],
       [-1.18810141,  1.27962673, -5.6727109 ],
       [-1.16849482,  1.26571214, -5.69752932],
       [-1.17340052,  1.29270089, -5.67240667],
       [-1.16556227,  1.32760322, -5.66247511],
       [-1.14595544,  1.3134464 , -5.68634319],
       [-1.16556227,  1.32760322, -5.66247511],
       [-1.14595544,  1.3134464 , -5.68634319],
       [-1.16556227,  1.32760322, -5.66247511],
       [-1.15576315,  1.36512983, -5.65491056],
       [-1.15576744,  1.40260279, -5.63520956],
       [-1.12930369,  1.45685077, -5.64564753],
       [-1.08422709,  1.5187639 , -5.61463499],
       [-1.0381633 ,  1.56513834, -5.61797142],
       [-1.05874956,  1.57739508, -5.59458828],
       [-1.0381633 ,  1.

# DIPY tuto [here](https://dipy.org/documentation/1.5.0/examples_built/streamline_registration/#example-streamline-registration)

In [26]:
import numpy as np
import nibabel as nib
from dipy.io.streamline import load_tractogram
from dipy.tracking.streamline import deform_streamlines

In [28]:
target_isocenter = np.diag(np.array([-0.7, 0.7, 0.7, 1]))

In [30]:
origin_affine = data.copy()
origin_affine[0][3] = -origin_affine[0][3]
origin_affine[1][3] = -origin_affine[1][3]
origin_affine[2][3] = origin_affine[2][3]/0.7
origin_affine[1][3] = origin_affine[1][3]/0.7**2

In [34]:
mni_streamlines = deform_streamlines(
    sft_streamlines, deform_field=transfo_dict['AffineTransform_float_3_3'][0:9].reshape((3, 3)),
    stream_to_current_grid=target_isocenter,
    current_grid_to_world=origin_affine, stream_to_ref_grid=target_isocenter,
    ref_grid_to_world=affine)

ValueError: Data needs to have 3 or 4 dimensions